In [27]:
import pandas as pd


In [28]:
# create dataframes

schools_df = pd.read_csv('resources/schools_complete.csv')
students_df = pd.read_csv('resources/students_complete.csv')

# merge into one dataframe 
merged_df = pd.merge(schools_df, students_df, on="school_name")


In [29]:
merged_df.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [30]:
# rename columns
merged_df = merged_df.rename(columns={"school_name": "School Name", 
                                    "type": "Type", "size": "Size", 
                                    "budget": "Budget", "student_name": "Student Name", "gender": "Gender",
                                    "grade": "Grade", "reading_score": "Reading Score", "math_score": "Math Score"})
merged_df.head()

,School ID,School Name,Type,Size,Budget,Student ID,Student Name,Gender,Grade,Reading Score,Math Score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [31]:
District Summary

SyntaxError: invalid syntax (3698797053.py, line 1)

In [32]:
# get unique school count
school_count = len(merged_df["School Name"].unique())
school_count

15

In [33]:
# get count of students
student_count = len(merged_df)
student_count

39170

In [46]:
# calculate total budget
#total_budget = merged_df["School Name"].unique().sum("Budget")
#total_budget

TypeError: 'str' object cannot be interpreted as an integer

In [35]:
# calculate average (mean) math score
average_math = merged_df["Math Score"].mean()
average_math

78.98537145774827

In [36]:
#calculate average (mean) reading score
average_reading = merged_df["Reading Score"].mean()
average_reading

81.87784018381414

In [37]:
# percentage passing math
passing_math_count = merged_df[(merged_df["Math Score"] >= 70)].count()["Student Name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [38]:
# percentage passing reading
passing_reading_count = merged_df[(merged_df["Reading Score"] >= 70)].count()["Student Name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [39]:
# calculate overall passing percentage
passing_math_reading_count = merged_df[
    (merged_df["Math Score"] >= 70) & (merged_df["Reading Score"] >= 70)
].count()["Student Name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [49]:
# create district summary snapshot
district_summary = pd.DataFrame({"Total Schools": [school_count],
                              "Total Students": student_count,
                              "Average Math Score": average_math,
                              "Average Reading Score": average_reading,
                                "% Passing Math": passing_math_percentage,
                                "% Passing Reading": passing_reading_percentage,
                                "Overall Passing %": overall_passing_rate})
# format columns
#district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
#district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

district_summary

,Total Schools,Total Students,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing %
0,15,39170,78.985371,81.87784,74.980853,85.805463,65.172326
